## Imports

In [77]:
import pandas as pd
import numpy as np
import json
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta

In [78]:
#Load excel data
exp = pd.json_normalize(pd.read_json('../data/exp.json')['expenses'])

In [79]:
exp

,title,type,cadence,date_added,first_due,amount
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0


In [80]:
#Check dtypes and update if needed
exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       9 non-null      object 
 1   type        9 non-null      object 
 2   cadence     9 non-null      object 
 3   date_added  9 non-null      object 
 4   first_due   9 non-null      object 
 5   amount      9 non-null      float64
dtypes: float64(1), object(5)
memory usage: 560.0+ bytes


In [81]:
#conver to datetime
exp['date_added'] = pd.to_datetime(exp['date_added'], format='%Y-%m-%d')
exp['first_due'] = pd.to_datetime(exp['first_due'], format='%Y-%m-%d')

exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       9 non-null      object        
 1   type        9 non-null      object        
 2   cadence     9 non-null      object        
 3   date_added  9 non-null      datetime64[ns]
 4   first_due   9 non-null      datetime64[ns]
 5   amount      9 non-null      float64       
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 560.0+ bytes


Nice!

Let's now calculate the next payment date. But first i'll add a count of the tracked months for reporting purposes.
https://stackoverflow.com/questions/30328427/add-months-to-a-datetime-column-in-pandas

In [82]:
#create a field that calculates the month in which you should've been saving since the last disbursement
exp['tracked_months'] = (((datetime.datetime.today().year-exp.date_added.dt.year)*12) + ((datetime.datetime.today().month-exp.date_added.dt.month))).astype(int)

In [83]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9


### Create a Last Disbursed Date Calculation

I'll do this with a while loop through each row

In [84]:
last_paid = [] #empty list for appending to

for index, row in exp.iterrows():
    payment_date = row['first_due']
    if payment_date >= datetime.datetime.today():
        last_paid.append(np.nan) 
    else:
        while (((datetime.datetime.today().year-payment_date.year)*12) + ((datetime.datetime.today().month-payment_date.month))) > row['cadence']:
            payment_date += relativedelta(months=row['cadence'])
        last_paid.append(payment_date)

In [85]:
exp['last_disbursed']  = last_paid

In [86]:
#update dtype to datetime for future formula usage
exp['last_disbursed'] = pd.to_datetime(exp['last_disbursed'], format='%m,%d,%Y')

In [87]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT


### Create a Next Due Date Calculation

I'll do this with a while loop through each row

In [88]:
due_dates = []
for index, row in exp.iterrows():
    next_date = row['first_due']
    while next_date < datetime.datetime.today():
        next_date += relativedelta(months=row['cadence'])
    due_dates.append(next_date)

In [89]:
exp['due_next'] = due_dates

In [90]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09


In [91]:
exp.dtypes

title                     object
type                      object
cadence                   object
date_added        datetime64[ns]
first_due         datetime64[ns]
amount                   float64
tracked_months             int64
last_disbursed    datetime64[ns]
due_next          datetime64[ns]
dtype: object

Now I'll calculate the months between next payment

In [92]:
#instead of months_till_due I'm going to make this the months to save
exp['months_to_save'] = np.where(exp['last_disbursed'].isnull(),
    (((exp['due_next'].dt.year - exp['date_added'].dt.year) * 12) + (exp['due_next'].dt.month - exp['date_added'].dt.month)),
    (((exp['due_next'].dt.year - exp['last_disbursed'].dt.year) * 12) + (exp['due_next'].dt.month - exp['last_disbursed'].dt.month))
).astype(int)

In [93]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11


Now calculate the monthly savings. There will be a differnce though for the items in which we save for the whole cadence vs. those we add with less time than the cadence. so I'll flag it if we have a full 12 months (as an example) to save vs. only 3 months left of the 12 in which we started saving. I think we need a "months to save" field

In [94]:
#updating this to not if false use months till due, but to the lesser of the cadence or the recalculated months from starting tracking to having to pay. Adding +1 to include the starting month.
#exp['months_to_save'] = exp['months_till_due']+1

In [95]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11


In [96]:
exp['monthly_sinking'] = round(exp['amount'] / exp['months_to_save'],2)
exp.replace([np.inf, -np.inf], np.nan, inplace=True)

In [97]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save,monthly_sinking
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13,2.31
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12,7.92
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12,20.42
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12,12.50
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12,25.00
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4,15.00
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4,15.00
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18,55.56
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11,181.82


Now I need to come up with a formula that calculates how much should be saved after your deposit this month based on how many months you should've saved since the last disbursement (or since you started tracking if no disbursement yet.)


In [98]:
#update to not include the current month.
#this -1 at the end will make it so the savings include the last portion on the month that the expense is used. For example if I'm saving for an oil change every 4 months, it will save 15 a month, with the savings growing from 45 to 60 in the month of the oil change.
exp['current_buildup'] = np.where(exp['last_disbursed'].isnull(),
                                  exp['tracked_months'],
                                  (((datetime.datetime.today().year-exp.last_disbursed.dt.year)*12) + ((datetime.datetime.today().month-exp.last_disbursed.dt.month))-1))

In [99]:
#convert to an integer format so it will round down to get the exact # of months.
#using fillna here to make it possible to convert if there are any NaNs
exp['current_buildup'] = exp['current_buildup'].fillna(0).astype(int)


In [100]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save,monthly_sinking,current_buildup
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13,2.31,10
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12,7.92,4
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12,20.42,2
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12,12.50,4
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12,25.00,2
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4,15.00,1
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4,15.00,3
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18,55.56,9
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11,181.82,9


Now we know how many months of savings we should have done since our last disbursement in each category so let's calculate what the total balance should be as of the last transfer to savings. I'll then calculate a final balance after transfer.

In [101]:
exp['exp_current_balance'] = exp['current_buildup'] * exp['monthly_sinking']

In [102]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save,monthly_sinking,current_buildup,exp_current_balance
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13,2.31,10,23.10
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12,7.92,4,31.68
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12,20.42,2,40.84
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12,12.50,4,50.00
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12,25.00,2,50.00
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4,15.00,1,15.00
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4,15.00,3,45.00
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18,55.56,9,500.04
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11,181.82,9,1636.38


Now that that's there, I want to create a column that calculates the disbursement (if there is one in the current month) as a negative value. I'll then combine that into a 'final_target' with expected_savings + monthly_needed - disbursement = final value.

In [103]:
#explored doing current balance as the disbursement, but that doesn't work with the using month of expense logic. Will need to make this dynamic if we offer a toggle.
exp['current_period_disburse'] = np.where((exp['due_next'].dt.month == datetime.datetime.today().month) & (exp['due_next'].dt.year == datetime.datetime.today().year),
                                          -exp['amount'],
                                          #-exp['exp_current_balance'],
                                          0)

In [104]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13,2.31,10,23.10,0.0
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12,7.92,4,31.68,0.0
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12,20.42,2,40.84,0.0
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12,12.50,4,50.00,0.0
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12,25.00,2,50.00,0.0
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4,15.00,1,15.00,0.0
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4,15.00,3,45.00,-60.0
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18,55.56,9,500.04,0.0
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11,181.82,9,1636.38,0.0


In [105]:
#sweet! Now I'll calculate the target ending balance

#update this to leave completed one-times as 0 target ending
exp['target_ending_balance'] = np.where((exp['type'] == 'One-Time') & ~(exp['last_disbursed'].isna()),
                                        0,
                                        exp['exp_current_balance'] + exp['monthly_sinking'] + exp['current_period_disburse'])

In [106]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse,target_ending_balance
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13,2.31,10,23.10,0.0,25.41
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12,7.92,4,31.68,0.0,39.60
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12,20.42,2,40.84,0.0,61.26
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12,12.50,4,50.00,0.0,62.50
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12,25.00,2,50.00,0.0,75.00
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4,15.00,1,15.00,0.0,30.00
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4,15.00,3,45.00,-60.0,0.00
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18,55.56,9,500.04,0.0,555.60
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11,181.82,9,1636.38,0.0,1818.20


In [107]:
#one more column for indiviual item activity
exp['current_month_activity'] = exp['target_ending_balance'] - exp['exp_current_balance']

In [108]:
exp

,title,type,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse,target_ending_balance,current_month_activity
0,Costco Membership,Recurring,12.0,2023-05-27,2024-06-30,30.0,10,NaT,2024-06-30,13,2.31,10,23.10,0.0,25.41,2.31
1,Chase Annual Fee,Recurring,12.0,2023-05-27,2023-10-31,95.0,10,2023-10-31,2024-10-31,12,7.92,4,31.68,0.0,39.60,7.92
2,Registration - Jess,Recurring,12.0,2023-05-27,2023-12-31,245.0,10,2023-12-31,2024-12-31,12,20.42,2,40.84,0.0,61.26,20.42
3,Registration - Brett,Recurring,12.0,2023-05-27,2023-10-31,150.0,10,2023-10-31,2024-10-31,12,12.50,4,50.00,0.0,62.50,12.50
4,Xmas Gifts,Recurring,12.0,2023-05-27,2023-12-31,300.0,10,2023-12-31,2024-12-31,12,25.00,2,50.00,0.0,75.00,25.00
5,Oil Change - Jess,Recurring,4.0,2023-05-27,2023-09-30,60.0,10,2024-01-30,2024-05-30,4,15.00,1,15.00,0.0,30.00,15.00
6,Oil Change - Brett,Recurring,4.0,2023-05-27,2023-07-31,60.0,10,2023-11-30,2024-03-30,4,15.00,3,45.00,-60.0,0.00,-45.00
7,New House Couch,One-Time,,2023-06-15,2024-12-31,1000.0,9,NaT,2024-12-31,18,55.56,9,500.04,0.0,555.60,55.56
8,Disney B-Day,One-Time,,2023-06-15,2024-05-09,2000.0,9,NaT,2024-05-09,11,181.82,9,1636.38,0.0,1818.20,181.82


In [32]:
exp[['title', 'cadence','date_added','first_due','amount','monthly_sinking']]

,title,cadence,date_added,first_due,amount,monthly_sinking
0,Costco Membership,12.0,2023-05-27,2024-06-30,30.0,2.31
1,Chase Annual Fee,12.0,2023-05-27,2023-10-31,95.0,7.92
2,Registration - Jess,12.0,2023-05-27,2023-12-31,245.0,20.42
3,Registration - Brett,12.0,2023-05-27,2023-10-31,150.0,12.50
4,Xmas Gifts,12.0,2023-05-27,2023-12-31,300.0,25.00
5,Oil Change - Jess,4.0,2023-05-27,2023-09-30,60.0,15.00
6,Oil Change - Brett,4.0,2023-05-27,2023-07-31,60.0,15.00
7,New House Couch,,2023-06-15,2024-12-31,1000.0,55.56
8,Disney B-Day,,2023-06-15,2024-05-09,2000.0,181.82


In [33]:
exp['monthly_sinking'].sum()

335.53

### Aggregation Steps

Now that we have that all built out, we need some formulas that calculate the aggregate totals, and give some output to users after taking some inputs.

In [34]:
#I want a copy of this with a new cooler name
sinking_funds = exp.copy()

### Add ability to say which bills are due this month.

In [35]:
due_this_month = sinking_funds[sinking_funds['current_period_disburse'] < 0][['title','due_next','amount']]

In [36]:
due_dictionary = due_this_month.set_index('title').T.to_dict('list')

In [37]:
due_dictionary

{'Oil Change - Brett': [Timestamp('2024-03-30 00:00:00'), 60.0]}

In [38]:
for key, value in due_dictionary.items():
    print(key, 'is due on', value[0].strftime('%m-%d-%Y'), ': $', str("{:.2f}".format(value[1])))

Oil Change - Brett is due on 03-30-2024 : $ 60.00


### Setup way for user input of their actual current balance.

This comes in handy when there is interest building up. That is a little bit less you have to save. Or for times in which you fall behind by a couple of months.

In [39]:
#define a function to check for a valid float input
def valid_float(message):
    while True:
        try:
            count = float(input(message))
            if count >= 0:
                return count
                break
        except:
            print('You must input a numerical value.\n' + message)

In [40]:
actual_beg_balance = valid_float("Enter Current Sinking Funds Balance: ")

### Create Calculated Features and Setup Test Prints

In [41]:
expected_beginning_balance = sinking_funds['exp_current_balance'].sum()
current_period_savings = sinking_funds['monthly_sinking'].sum()
current_period_disbursements = sinking_funds['current_period_disburse'].sum()
net_period_activity = sinking_funds['current_month_activity'].sum()
target_ending_balance = sinking_funds['target_ending_balance'].sum()
actual_period_activity = target_ending_balance - actual_beg_balance
balance_adj = expected_beginning_balance - actual_beg_balance

In [42]:
print("You're expected current balance is: $" + str("{:.2f}".format(expected_beginning_balance)) + ".")
print("You're actual current balance is: $" + str("{:.2f}".format(actual_beg_balance)) + ".\n")
print('---'*10)
if actual_period_activity < 0:
    print("You should move $" + str("{:.2f}".format(-actual_period_activity)) + " from your sinking funds balance to your bill-paying account.")
else:
    print("You should move $" + str("{:.2f}".format(actual_period_activity)) + " from your checking account to your sinking funds balance.")
print("This is made up of: \n- $" + str("{:.2f}".format(current_period_savings)) + " savings for future expenses\n- $" + str("{:.2f}".format(-current_period_disbursements)) + " of disbursements for current month expense\n- $"+ str("{:.2f}".format(balance_adj)) + " of balance adjustments.\n")
print('---'*10)

print("With these changes, your target ending balance will be: $" + str("{:.2f}".format(target_ending_balance)) + ".")


You're expected current balance is: $2392.04.
You're actual current balance is: $2250.00.

------------------------------
You should move $417.57 from your checking account to your sinking funds balance.
This is made up of: 
- $335.53 savings for future expenses
- $60.00 of disbursements for current month expense
- $142.04 of balance adjustments.

------------------------------
With these changes, your target ending balance will be: $2667.57.


In [143]:
print("-"*5 + "USER SUMMARY" + "-"*5)

-----USER SUMMARY-----


# V2 Testing
I want to remove excel from the equation and actually have this read/write from/to a json file. To do this we'll need to write functions for adding and removing records using user inputs.

Might pivot to a sqllite database for this. If I do go json, I either need to update the key and then have a subset with teh data. or have the title in the full layer with all the data and be able to add another line to this.


In [144]:
#establish possible inputs and alert
input_options = ['A','B','C','D','E']
alert = "Option not available."

In [145]:
def valid_select(message, n_choices):
    """Function for testing valid user input."""
    while True:
        try:
            answer = input(message).upper()
            if answer in input_options[:n_choices]:
                return answer
                break
            else:
                print(alert + ' Try again: ')
        except:
            print(alert + ' Try again: ')

In [146]:
#valid_select("What would you like to do?\nA) View Current Month Report\nB) Add New Expense\nC) Delete an Expense", 4)

In [147]:
def user_input_text(message):
    "Function for accepting user input of text and savings to a variable"
    var = str(input(message))
    return var

In [148]:
def valid_date(date_title):
    "Function for checking for valid date input"
    while True:
        try:
            year, month, day = map(int, input('Enter the ' + date_title + ' in YYYY-MM-DD format: ').split('-'))
            input_date = datetime.date(year, month, day)
            return input_date.strftime('%Y-%m-%d')
            break
        except:
            print('Please enter in YYYY-MM-DD format: ')
    

In [149]:
#valid_date('Due Date')

### Test specifc reading

In [150]:
#test reading
with open('../data/exp.json', 'r') as f:
    data = json.load(f)

[item for item in data['expenses'] if item['title'] == 'Costco']

[]

Might be able to use this code to get specific items
[d for d in a if d['name'] == 'pluto']


https://stackoverflow.com/questions/4391697/find-the-index-of-a-dict-within-a-list-by-matching-the-dicts-value

https://stackoverflow.com/questions/4391697/find-the-index-of-a-dict-within-a-list-by-matching-the-dicts-value

### Test Writing New Object

In [151]:
#sample varible with new entry
new_entry = {'title': 'Xmas Gifts',
 'cadence': '12',
 'date_added': '5/27/2023',
 'first_due': '12/31/2023',
 'amount': '600'}

In [152]:
#test writing to json
with open('../data/exp.json') as f:
    data = json.load(f)
    temp = data["expenses"]
    temp.append(new_entry)
    with open('../data/exp.json', 'w') as f:
        json.dump(data, f)

In [75]:
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'Costco Membership',
  'type': 'Recurring',
  'cadence': 12.0,
  'date_added': '2023-05-27',
  'first_due': '2024-06-30',
  'amount': 30.0},
 {'title': 'Chase Annual Fee',
  'type': 'Recurring',
  'cadence': 12.0,
  'date_added': '2023-05-27',
  'first_due': '2023-10-31',
  'amount': 95.0},
 {'title': 'Registration - Jess',
  'type': 'Recurring',
  'cadence': 12.0,
  'date_added': '2023-05-27',
  'first_due': '2023-12-31',
  'amount': 245.0},
 {'title': 'Registration - Brett',
  'type': 'Recurring',
  'cadence': 12.0,
  'date_added': '2023-05-27',
  'first_due': '2023-10-31',
  'amount': 150.0},
 {'title': 'Xmas Gifts',
  'type': 'Recurring',
  'cadence': 12.0,
  'date_added': '2023-05-27',
  'first_due': '2023-12-31',
  'amount': 300.0},
 {'title': 'Oil Change - Jess',
  'type': 'Recurring',
  'cadence': 4.0,
  'date_added': '2023-05-27',
  'first_due': '2023-09-30',
  'amount': 60.0},
 {'title': 'Oil Change - Brett',
  'type': 'Recurring',
  'cadence': 4.0,
  'date_added':

### Test updating specific value
As a test we'll update our Xmas Gift fund to $500 instead of $600.

In [154]:
#finding index #
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']
    index = next((index for (index, d) in enumerate(temp) if d['title'] == 'Xmas Gifts'), None)

index

3

In [155]:
#test saving down index # and writing to
with open('../data/exp.json') as f:
    data = json.load(f)
    temp = data["expenses"]
    index = next((index for (index, d) in enumerate(temp) if d['title'] == 'Xmas Gifts'), None)
    temp[index]['amount'] = 500
    with open('../data/exp.json', 'w') as f:
        json.dump(data, f)

In [156]:
#view if that worked
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'XMas Gifts',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 600.0},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Sunny',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 10000.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

### Test Deleting Item

In [157]:
#test with read
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']
    temp[:] = [d for d in temp if d.get('title') != 'Costco']

temp

[{'title': 'XMas Gifts',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 600.0},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Sunny',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 10000.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

In [158]:
#test actual delete

#test saving down index # and writing to
with open('../data/exp.json') as f:
    data = json.load(f)
    temp = data["expenses"]
    temp[:] = [d for d in temp if d.get('title') != 'Costco']
    with open('../data/exp.json', 'w') as f:
        json.dump(data, f)

In [159]:
#view if that worked
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'XMas Gifts',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 600.0},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Sunny',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 10000.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

## Define JSON Functions

In [160]:
#sample varible with new entry
new_entry = {'title': 'Xmas Gifts',
 'cadence': '12',
 'date_added': '5/27/2023',
 'first_due': '12/31/2023',
 'amount': '600'}

In [161]:
title = "Xmas Gifts"
cadence = 12
date_add = datetime.date.today().strftime('%Y-%m-%d')
first_due = datetime.date(2023,12,31).strftime('%Y-%m-%d')
amount = 600

In [162]:
exp_dict = {"title": title, "cadence": cadence, "date_added": date_add, "first_due": first_due, "amount": amount}
exp_dict

{'title': 'Xmas Gifts',
 'cadence': 12,
 'date_added': '2023-06-03',
 'first_due': '2023-12-31',
 'amount': 600}

#### Adding an expense item

In [169]:
def get_items(json_path, json_var):
    "Take in json variables and return the list of titles to reference in other functions."
    #create list of items saved to data
    with open(json_path) as f:
        data = json.load(f)
        temp = data[json_var]
        item_list = [list(dict.values())[0] for dict in temp]
    return item_list

In [183]:
def exp_in_list(message, list, match_wanted):
    "Take a user input after a prompt and return TRUE/FALSE if it is in a given list or not"
    while True:
        var = str(input(message))
        if match_wanted == False:
            if var in list:
                print('This expense already exists, please choose another title: ')
            else:
                return var
                break
        else:
            if var in list:
                return var
                break
            else:
                print('Expense not in list, please enter a valid expense: ')

In [184]:
def add_exp(json_path, json_var):
    "Take user inputs, create a dictionary, and write that dictionary to a json file."

    #setup variables for input
    title = exp_in_list("Expense Title: ", get_items(json_path, json_var), False) #validate the input isn't already in the list
    cadence = valid_float("Monthly cadence (every x months): ")
    start_date = datetime.date.today().strftime('%Y-%m-%d')
    first_due = valid_date("Due Date")
    amount = valid_float("How much do you want to save?: ")
    
    #create dictionary from variables
    exp_dict = {"title": title, "cadence": cadence, "date_added": start_date, "first_due": first_due, "amount": amount}

    with open(json_path) as f:
        data = json.load(f)
        temp = data[json_var]
        temp.append(exp_dict)
        with open(json_path, 'w') as f:
            json.dump(data, f)

In [175]:
#add_exp('../data/exp.json', 'expenses')

This expense already exists, please choose another title: 


#### Updating an Item

In [166]:
#valid_item('../data/exp.json', 'expenses', "Which expense would you like to update?: \n")

In [185]:
def update_exp(json_path, json_var):
    "Take user inputs and update the value of their choice."
    #user input validated against list of itmes
    item_to_update = exp_in_list("Which expense would you like to update?: \n",get_items(json_path, json_var), True)

    #user input on which field to update
    field_choice = valid_select("Which field would you like to update?:\nA) Title\nB) Cadence\nC) Date Added\nD) First Due\nE) Amount", 5)
    if field_choice == 'A':
        field = 'title'
    elif field_choice == 'B':
        field = 'cadence'
    elif field_choice == 'C':
        field = 'date_added'
    elif field_choice == 'D':
        field = 'first_due'
    elif field_choice == 'E':
        field = 'amount'

    #user input to update field, using an if statement to decice which input validation to use
    if field == ('title'):
        new_value = user_input_text("New Title: ")
    elif field in ('cadence', 'amount'):
        new_value = valid_float("New Value: ")
    else:
        new_value = valid_date("New Date: ")
    
    #write to JSON
    with open(json_path) as f:
        data = json.load(f)
        temp = data[json_var]
        index = next((index for (index, d) in enumerate(temp) if d['title'] == item_to_update), None)
        temp[index][field] = new_value
        with open('../data/exp.json', 'w') as f:
            json.dump(data, f)

In [182]:
#update_exp('../data/exp.json', 'expenses')

Expense not in list, please enter a valid expense: 


### Deleting and Item

In [194]:
def delete_expense(json_path, json_var):
    item_to_delete = exp_in_list("Which expense would you like to delete?: \n",get_items(json_path, json_var), True)
    
    with open(json_path, 'r') as f:
        data = json.load(f)
        temp = data[json_var]
        temp[:] = [d for d in temp if d.get('title') != item_to_delete]
        with open(json_path, 'w') as f:
            json.dump(data, f)

In [188]:
#delete_expense('../data/exp.json', 'expenses')

Expense not in list, please enter a valid expense: 


## Test Full Sequence of Functions

In [190]:
#add_exp('../data/exp.json', 'expenses')

In [191]:
#add_exp('../data/exp.json', 'expenses')

In [192]:
#update_exp('../data/exp.json', 'expenses')

In [193]:
#delete_expense('../data/exp.json', 'expenses')

## Read from JSON into DF

In [265]:
test_df = pd.json_normalize(pd.read_json('../data/exp.json')['expenses'])

### Print Active Expenses

In [266]:
def view_expenses(df, index_col):
    print("Active Expenses: \n")
    if len(df) == 0:
        print('There are no active expenses.')
    else:
        print(df.set_index(index_col))

In [267]:
len(test_df)

0

In [272]:
if test_df.empty == True:
    print(1)
else:
    print(0)

1


In [268]:
view_expenses(test_df, 'title')

Active Expenses: 

There are no active expenses.


## Main Menu Options Test

In [200]:
#just a test of the logic, will change field variable setting to running functions in production fiel

menu_choice = valid_select("What function would you like to perform?:\nA) Run Monthly Report\nB) View Active Expenses\nC) Add New Expense\nD) Update Expense Item\nE) Delete Expense", 5)
if menu_choice == 'A':
    test = 'title'
elif menu_choice == 'B':
    test = 'cadence'
elif menu_choice == 'C':
    test = 'date_added'
elif menu_choice == 'D':
    test = 'first_due'
elif menu_choice == 'E':
    test = 'amount'

In [201]:
test

'title'